In [1]:
import caffe

from caffe import layers as L, params as P, to_proto

from caffe.proto import caffe_pb2

from caffe.coord_map import crop

import copy

In [2]:
def conv_relu(bottom, ks, nout, stride=1, pad=0, group=1):
    conv = L.Convolution(bottom, kernel_size=ks, stride=stride,
                                num_output=nout, pad=pad, group=group, weight_filler={"type":"msra"})
    return L.ReLU(conv, in_place=True)

def fc_relu(bottom, nout):
    fc = L.InnerProduct(bottom, num_output=nout)
    return fc, L.ReLU(fc, in_place=True)

def max_pool(bottom, ks=2, stride=2):
    return L.Pooling(bottom, pool=P.Pooling.MAX, kernel_size=ks, stride=stride)

def deconv_relu(bottom, ks, nout, stride=1, pad=0, group=1, crop=0):
    convolution_param = dict(num_output=nout, kernel_size=ks, stride=stride,pad=pad, group=group,
            bias_term=False)
    deconv = L.Deconvolution(bottom,convolution_param=convolution_param)
    return L.ReLU(deconv, in_place=True)
    
    
    

def get_blob_shape(blob):
    _,fn = write_to_file(blob,filename="tmp")
    net = caffe.Net(fn, caffe.TEST)
    blob_key = net.blobs.keys()[-1]
    blob_obj = net.blobs[blob_key]
    blob_shape = [blob_obj.shape[i] for i in range(len(blob_obj.shape))]
    return blob_shape
    
    
def convert_layer_to_prototxt(layer):
    return str(to_proto(layer))

def write_prototxt_str(filename, prototxt_str):
    #print prototxt_str
    with open(filename, 'w') as f:
        f.write(prototxt_str)

def write_to_file(layer,filename="train"):
    
    prototxt_str = convert_layer_to_prototxt(layer)

    write_prototxt_str(filename + ".prototxt", prototxt_str)
    
    return prototxt_str, filename + ".prototxt"

In [4]:
net_vars = ["PRECT",
"PS",
"PSL",
"QREFHT",
"T200",
"T500",
"TMQ",
"TREFHT",
"TS",
"U850",
"UBOT",
"V850",
"VBOT",
"Z1000",
"Z200",
"ZBOT"]

In [3]:
batch_size=1
CAFFE=1
SUM=1

def make_netcdf_network(inp_x = 768,inp_y=1152,lbl_ch = (4,4,2), name="netcdf"):
    data = L.NetCDFData(source=,
                       variable_data=net_vars, variable_label=["xmin"], batch_size=2)
    conv = conv_relu(data, ks=5, nout=8, pad=2, stride=2)
    pstr, fn = write_to_file(conv, name)
    return pstr, fn, conv
    
    


def make_network(inp_x = 768,inp_y=1152,lbl_ch = (4,4,2), name="sm_sq_2d"):
    data = L.DummyData(shape={'dim':[batch_size,16,inp_x,inp_y]}, data_filler={"type":"gaussian", "mean":0, "std":1})
    lbl_x = inp_x / 64
    lbl_y = inp_y / 64
    label1 = L.DummyData(shape={'dim':[batch_size,lbl_ch[0],lbl_x, lbl_y]}, data_filler={"type":"gaussian", "mean":0, "std":1})
    label2 = L.DummyData(shape={'dim':[batch_size,lbl_ch[1],lbl_x, lbl_y]}, data_filler={"type":"gaussian", "mean":0, "std":1})
    label3 = L.DummyData(shape={'dim':[batch_size,lbl_ch[2],lbl_x, lbl_y]}, data_filler={"type":"gaussian", "mean":0, "std":1})
    nfilters_list = [128, 256, 512, 768, 1024, 1280]
    #nfilters_list = [f/2 for f in nfilters_list ]
    conv = data

    num_layers = 6
    encoder_blobs = [data]
    for i in range(num_layers):
        nfilters = nfilters_list[i]
        conv = conv_relu(conv, ks=5, nout=nfilters, pad=2, stride=2)
        encoder_blobs.append(conv)

    nfilters_list.pop()
    nfilters_list.reverse()
    encoder_blobs.pop()
    encoder_blobs.reverse()
    nfilters_list.append(16)

    encoder = conv
    print get_blob_shape(encoder)
    for i in range(num_layers):
        nfilters = nfilters_list[i]
        conv = deconv_relu(conv,5, nfilters, stride=2)
        conv = L.Crop(conv,encoder_blobs[i], axis=2,offset=1)
        #print get_blob_shape(conv)

    rec = conv

    conv1 = conv_relu(encoder,ks=3,pad=1,nout=lbl_ch[0])

    conv2 = L.Convolution(encoder,kernel_size=3,pad=1, stride=1, num_output=lbl_ch[1])

    conv2 = L.Sigmoid(conv2)
    conv3 = L.Convolution(encoder,kernel_size=3,pad=1,stride=1, num_output=lbl_ch[2])

    conv3 = L.Sigmoid(conv3)
    yolo = L.Concat(conv1,conv2, conv3)
    lbl = L.Concat(label1, label2, label3)
    loss1 = L.EuclideanLoss(yolo, lbl)
    lossr = L.EuclideanLoss(rec,data)
    final_loss = L.Eltwise(loss1,lossr,operation=SUM)
    pstr, fn = write_to_file(final_loss, name)
    return pstr, fn, final_loss

In [6]:
pstr, fn, fl= make_netcdf_network()

NameError: global name 'TMQ' is not defined

In [4]:
# name = "sm_sq_2d_16"
# pstr, fn, fl = make_network(inp_x=768,inp_y=768,lbl_ch=(16,16,16), name=name)

[1, 1280, 12, 12]


In [ ]:
#test
net=caffe.Net(fn,caffe.TRAIN)

net.forward()

net.backward()

In [6]:
#add engine caffe
def add_engine_caffe(pstr, fn):
    splits = pstr.split("operation: SUM")
    newstr=''.join(splits[:-1] + ["operation: SUM\n    engine: CAFFE"] +[splits[-1]])
    write_prototxt_str(fn, newstr)
    return newstr

In [7]:
newstr= add_engine_caffe(pstr, fn)

In [8]:
print newstr

layer {
  name: "DummyData1"
  type: "DummyData"
  top: "DummyData1"
  dummy_data_param {
    data_filler {
      type: "gaussian"
      mean: 0
      std: 1
    }
    shape {
      dim: 1
      dim: 16
      dim: 768
      dim: 768
    }
  }
}
layer {
  name: "Convolution1"
  type: "Convolution"
  bottom: "DummyData1"
  top: "Convolution1"
  convolution_param {
    num_output: 128
    pad: 2
    kernel_size: 5
    group: 1
    stride: 2
    weight_filler {
      type: "msra"
    }
  }
}
layer {
  name: "ReLU1"
  type: "ReLU"
  bottom: "Convolution1"
  top: "Convolution1"
}
layer {
  name: "Convolution2"
  type: "Convolution"
  bottom: "Convolution1"
  top: "Convolution2"
  convolution_param {
    num_output: 256
    pad: 2
    kernel_size: 5
    group: 1
    stride: 2
    weight_filler {
      type: "msra"
    }
  }
}
layer {
  name: "ReLU2"
  type: "ReLU"
  bottom: "Convolution2"
  top: "Convolution2"
}
layer {
  name: "Convolution3"
  type: "Convolution"
  bottom: "Convolution2"
  t